In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
import IPython
import mlflow

In [2]:
!pwd

/c/Users/yacin/Desktop/DATA SCIENCE OPENCLASSROOMS/Projet 13/notebook


In [3]:
calendar = pd.read_csv('../datasets/calendar.csv', sep=',')

In [17]:
sales = pd.read_csv('../datasets/sales_train_validation.csv', sep=',')

In [5]:
sell_prices = pd.read_csv('../datasets/sell_prices.csv', sep=',')

In [6]:
calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          1969 non-null   object
 1   wm_yr_wk      1969 non-null   int64 
 2   weekday       1969 non-null   object
 3   wday          1969 non-null   int64 
 4   month         1969 non-null   int64 
 5   year          1969 non-null   int64 
 6   d             1969 non-null   object
 7   event_name_1  162 non-null    object
 8   event_type_1  162 non-null    object
 9   event_name_2  5 non-null      object
 10  event_type_2  5 non-null      object
 11  snap_CA       1969 non-null   int64 
 12  snap_TX       1969 non-null   int64 
 13  snap_WI       1969 non-null   int64 
dtypes: int64(7), object(7)
memory usage: 215.5+ KB


In [18]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1919 entries, id to d_1913
dtypes: int64(1913), object(6)
memory usage: 446.4+ MB


In [8]:
sell_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6841121 entries, 0 to 6841120
Data columns (total 4 columns):
 #   Column      Dtype  
---  ------      -----  
 0   store_id    object 
 1   item_id     object 
 2   wm_yr_wk    int64  
 3   sell_price  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 208.8+ MB


In [9]:
#Exploration des données 

In [10]:
calendar[['date', 'year', 'month', 'weekday', 'wday']].head()

,date,year,month,weekday,wday
0,2011-01-29,2011,1,Saturday,1
1,2011-01-30,2011,1,Sunday,2
2,2011-01-31,2011,1,Monday,3
3,2011-02-01,2011,2,Tuesday,4
4,2011-02-02,2011,2,Wednesday,5


In [14]:
calendar[['event_name_1', 'event_type_1']].dropna().head()

,event_name_1,event_type_1
8,SuperBowl,Sporting
16,ValentinesDay,Cultural
23,PresidentsDay,National
39,LentStart,Religious
46,LentWeek2,Religious


In [16]:
calendar['date'].duplicated().sum()

np.int64(0)

In [19]:
sales[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']].head()

,id,item_id,dept_id,cat_id,store_id,state_id
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA


In [20]:
sales.filter(regex="^d_").shape

(30490, 1913)

In [21]:
sales['state_id'].value_counts()

state_id
CA    12196
TX     9147
WI     9147
Name: count, dtype: int64

In [22]:
sales.filter(regex="^d_").sum(axis=1).describe()

count     30490.000000
mean       2154.654280
std        5217.226438
min          10.000000
25%         355.000000
50%         848.000000
75%        2031.750000
max      250502.000000
dtype: float64

le volume total vendu sur 1913 jours pour chaque couple produit × magasin

mean ≈ 2 155

En moyenne, un produit-magasin a vendu ~2 155 unités sur 5 ans

Soit ≈ 1,1 unité par jour
✔ Très réaliste pour du retail

In [31]:
sales_ca = sales[sales['state_id'] == 'CA']

In [32]:
sales_ca.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12196 entries, 0 to 12195
Columns: 1919 entries, id to d_1913
dtypes: int64(1913), object(6)
memory usage: 178.7+ MB


In [33]:
sales_columns = sales_ca.filter(regex="^d_")

sales_columns.shape

(12196, 1913)

In [35]:
sales_columns.head()

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,0,0,0,0,0,0,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,0,0,0,0,0,0,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,0,0,0,0,0,0,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [36]:
daily_sales_ca = sales_columns.sum(axis=0).reset_index()
daily_sales_ca.columns = ["d", "sales"]

daily_sales_ca.head()

,d,sales
0,d_1,14195
1,d_2,13805
2,d_3,10108
3,d_4,11047
4,d_5,9925


In [37]:
daily_sales_ca.describe()

,sales
count,1913.000000
mean,14989.831155
std,3257.223505
min,5.000000
25%,12834.000000
50%,14678.000000
75%,16846.000000
max,25224.000000


### Agrégation des ventes en Californie

Les ventes ont été filtrées sur l’État de Californie, puis agrégées par jour afin d’obtenir une série temporelle unique représentant le volume total des ventes quotidiennes. Cette agrégation permet de réduire la variabilité liée aux produits individuels et de faciliter la modélisation.


In [38]:
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [39]:
calendar_cols = [
    "d",
    "date",
    "wday",
    "month",
    "year",
    "event_name_1",
    "event_name_2"
]

calendar_small = calendar[calendar_cols]
calendar_small.head()

,d,date,wday,month,year,event_name_1,event_name_2
0,d_1,2011-01-29,1,1,2011,NaN,NaN
1,d_2,2011-01-30,2,1,2011,NaN,NaN
2,d_3,2011-01-31,3,1,2011,NaN,NaN
3,d_4,2011-02-01,4,2,2011,NaN,NaN
4,d_5,2011-02-02,5,2,2011,NaN,NaN


In [41]:
daily_sales_ca = daily_sales_ca.merge(
    calendar_small,
    on = 'd',
    how='left'
)

In [52]:
daily_sales_ca

,d,sales,date,wday,month,year,event_name_1,event_name_2,is_weekend,is_event
0,d_1,14195,2011-01-29,1,1,2011,NaN,NaN,1,0
1,d_2,13805,2011-01-30,2,1,2011,NaN,NaN,1,0
2,d_3,10108,2011-01-31,3,1,2011,NaN,NaN,0,0
3,d_4,11047,2011-02-01,4,2,2011,NaN,NaN,0,0
4,d_5,9925,2011-02-02,5,2,2011,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...
1908,d_1909,15148,2016-04-20,5,4,2016,NaN,NaN,0,0
1909,d_1910,14488,2016-04-21,6,4,2016,NaN,NaN,0,0
1910,d_1911,17095,2016-04-22,7,4,2016,NaN,NaN,0,0
1911,d_1912,21834,2016-04-23,1,4,2016,NaN,NaN,1,0


In [44]:
daily_sales_ca.isna().sum()

d                  0
sales              0
date               0
wday               0
month              0
year               0
event_name_1    1759
event_name_2    1909
dtype: int64

In [45]:
daily_sales_ca['date'] = pd.to_datetime(daily_sales_ca['date'])

In [54]:
daily_sales_ca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1913 entries, 0 to 1912
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   d           1913 non-null   object        
 1   sales       1913 non-null   int64         
 2   date        1913 non-null   datetime64[ns]
 3   wday        1913 non-null   int64         
 4   month       1913 non-null   int64         
 5   year        1913 non-null   int64         
 6   is_weekend  1913 non-null   int64         
 7   is_event    1913 non-null   int64         
dtypes: datetime64[ns](1), int64(6), object(1)
memory usage: 119.7+ KB


### Jointure avec le calendrier

Les ventes quotidiennes agrégées ont été enrichies à l’aide du fichier `calendar.csv`, permettant d’associer chaque jour de vente à une date réelle ainsi qu’à des informations calendaires (jour de la semaine, mois, année, événements). Cette étape est essentielle pour capturer les effets de saisonnalité dans la modélisation.

In [47]:
daily_sales_ca["is_weekend"] = daily_sales_ca["wday"].isin([1, 2]).astype(int)

In [50]:
daily_sales_ca["is_event"] = (
    daily_sales_ca["event_name_1"].notna() |
    daily_sales_ca["event_name_2"].notna()
).astype(int)

In [53]:
daily_sales_ca.drop(
    columns=["event_name_1", "event_name_2"],
    inplace=True
)

### Création des features temporelles

Des variables temporelles simples ont été créées à partir du calendrier afin de capturer les effets de saisonnalité et de comportement d’achat. Ces variables sont connues à l’avance et facilement interprétables par les équipes métier.

In [56]:
daily_sales_ca = daily_sales_ca.sort_values("date").reset_index(drop=True)

In [61]:
daily_sales_ca

,d,sales,date,wday,month,year,is_weekend,is_event,sales_lag_7,sales_rolling_mean_7
0,d_1,14195,2011-01-29,1,1,2011,1,0,NaN,NaN
1,d_2,13805,2011-01-30,2,1,2011,1,0,NaN,NaN
2,d_3,10108,2011-01-31,3,1,2011,0,0,NaN,NaN
3,d_4,11047,2011-02-01,4,2,2011,0,0,NaN,NaN
4,d_5,9925,2011-02-02,5,2,2011,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1908,d_1909,15148,2016-04-20,5,4,2016,0,0,14614.0,17557.714286
1909,d_1910,14488,2016-04-21,6,4,2016,0,0,15069.0,17474.714286
1910,d_1911,17095,2016-04-22,7,4,2016,0,0,16255.0,17594.714286
1911,d_1912,21834,2016-04-23,1,4,2016,1,0,20564.0,17776.142857


In [58]:
daily_sales_ca["sales_lag_7"] = daily_sales_ca["sales"].shift(7)

In [60]:
daily_sales_ca["sales_rolling_mean_7"] = (
    daily_sales_ca["sales"]
    .rolling(window=7)
    .mean()
)